# Usage Instructions
1. Place the receipt image in the same folder as the code.
2. Enter the image path in the image_path field of the entry point.
3. Run all the code cells in order.

Note: The first time you run this code, you need to install the required libraries.

## Expected Result
1. The program will recognize the items and prices from the receipt image.
2. A visual interactive viewer will appear.
3. You can add, update, delete, or export items using the buttons.

In [387]:
#!pip install opencv-python paddlepaddle paddleocr pillow ipywidgets

In [388]:
import cv2
import numpy as np
import pandas as pd
import re
import os
from paddleocr import PaddleOCR
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import warnings

warnings.filterwarnings('ignore') #Ignore all Python warnings

print("All libraries imported successfully!")

All libraries imported successfully!


# PART 1: Image Preprocessing Functions

In [389]:
# Load image from file
def load_image(image_path):
    if not os.path.exists(image_path):
        print(f"❌ Error: File not found - {image_path}")
        return None
    
    image = cv2.imread(image_path)
    if image is None:
        print("❌ Error: Cannot read image")
        return None
    
    return image

# Complete image preprocessing pipeline
def preprocess_image(image_path):

    # Load image
    original = load_image(image_path)
    if original is None:
        return None
    
    # Convert to grayscale
    gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    
    # Remove noise
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
    
    # Apply adaptive thresholding (convert to black & white)
    threshold = cv2.adaptiveThreshold(
        denoised, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )
    
    return threshold


# PART 2: OCR Recognition Functions

In [390]:
#Initialize PaddleOCR engine
def initialize_ocr():
    ocr = PaddleOCR(use_angle_cls=True, lang='en')
    return ocr

#Perform OCR on image
def perform_ocr(ocr, image_path):
    
    #"Get all text from OCR
    result = ocr.ocr(image_path)
    
    if result is None or len(result) == 0:
        print("❌ OCR returned no results!")
        return []
    
    #Print the OCR result
    #print(result)
    #print("="*70 + "\n")
    return result[0] if len(result) > 0 else []

#Organize OCR results
def organize_ocr_results(ocr_result):
    df_ocr_result = pd.DataFrame(columns=["text","confidence","left_top_x","left_top_y",
                                          "right_top_x","right_top_y","left_bottom_x",
                                          "left_bottom_y","right_bottom_x","right_bottom_y"])
    
    print("Organize OCR data...")

    texts = ocr_result.get('rec_texts', [])
    scores = ocr_result.get('rec_scores', [1.0] * len(texts))
    polys = ocr_result.get('rec_polys', [])

    
    for idx, text in enumerate(texts):
        confidence = scores[idx] if idx < len(scores) else 1.0   #Use 1.0 when scores and text lengths differ
        
        # Get coordinates
        if idx < len(polys) and len(polys[idx]) >= 4:
            poly = polys[idx]
        else:
            poly = [[0,0],[0,0],[0,0],[0,0]]

        # Save text into dataframe
        df_ocr_result.loc[len(df_ocr_result.index)] = [str(text),float(confidence),poly[0][0],poly[0][1],
                                                       poly[1][0],poly[1][1],poly[2][0],poly[2][1],poly[3][0],
                                                       poly[3][1]]
    return df_ocr_result

# PART 3: Information Extraction Functions

In [391]:
#These functions must be called in order

#Extract date from text
def extract_date(text):
    """
    Extract a date from text and return it in MM/DD/YYYY format.
    If no date found, return "0".
    """
    # Various date matching patterns
    patterns = [
        r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}',        # 25/12/2025 or 12-25-25
        r'\d{4}[-/]\d{1,2}[-/]\d{1,2}',          # 2025/12/25
        r'\d{1,2}\.\d{1,2}\.\d{2,4}',            # 25.12.2025
        r'\d{1,2}\s+\w+\s+\d{2,4}',              # 25 Dec 2025
        r'\w+\s+\d{1,2},?\s+\d{2,4}',            # Dec 25, 2025
        r'\d{1,2}(st|nd|rd|th)?\s+\w+\s+\d{4}',  # 25th December 2025
    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            found_date = match.group()

            # Try to parse the date and return in MM/DD/YYYY format
            for fmt in ["%d/%m/%Y", "%m/%d/%Y", "%Y/%m/%d", "%d-%m-%Y", "%m-%d-%Y", "%d.%m.%Y",
                        "%d %b %Y", "%d %B %Y", "%b %d, %Y", "%B %d, %Y", "%d %B %Y", "%d %b %Y"]:
                try:
                    dt = datetime.strptime(found_date, fmt)
                    return dt.strftime("%m/%d/%Y")
                except:
                    continue
            # If matched but cannot be parsed, return the original string
            return found_date
    # Return "0" if no match is found
    return 0
    
#====================================================================================================
def classify_text_type(text):
    """
    Classify text type
    
    Returns:
    - 'price' : Price (two decimal places)
    - 'code' : Code (long digit string or alphanumeric)
    - 'header' : Header keyword (PRICE, QTY, AMOUNT, etc.)
    - 'payment' : Payment keyword (TOTAL, CASH, CHANGE, etc.)
    - 'invalid': All symbols or over 80% symbols
    - 'text' : Regular text

    """  
    
    text_clean = text.strip()
    text_lower = text_clean.lower()


    # 1. Check for date type
    if extract_date(text) != 0:
        return 'date'
    
    
    # 2. Check for header keywords
    header_keywords = ['price', 'amount', 'qty', 'quantity', 'item', 
                      'description', 'product', 'code', 'desc']
    
    if any(kw == text_lower or kw in text_lower for kw in header_keywords):
        if len(text_clean) < 20:  # Header keywords are usually short
            return 'header'
    
    # 3. Check for payment keywords
    payment_keywords = ['total', 'subtotal', 'grand total', 
                       'cash', 'payment', 'paid',
                       'change', 'balance',
                       'tax', 'vat', 'gst',
                       'discount', 'promo']
    
    if any(kw in text_lower for kw in payment_keywords):
        return 'payment'
    
    # 4. Check if it's a price (number with 2 decimals)
    # Pattern: $12.50 or 12.50 or 12.50$
    price_pattern = r'^\$?\s*\d+[.,]\d{2}\s*\$?$'
    if re.match(price_pattern, text_clean):
        return 'price'
    
    # 5. Check if it is a product code
    text_no_space = text_clean.replace(' ', '').replace('-', '').replace('.', '')
    
    if len(text_no_space) > 0:
        # Rule 1: Pure digits with length > 3 (4 or more digits)
        if text_no_space.isdigit() and len(text_no_space) > 3:
            return 'code'
        
        # Rule 2: Digit ratio > 70% 
        # Also require minimum length > 3 to avoid short mixed text
        digit_count = sum(c.isdigit() for c in text_no_space)
        total_length = len(text_no_space)
        digit_ratio = digit_count / total_length
        
        if digit_ratio > 0.7 and total_length > 3:
            return 'code'

     # 6. Check for symbols
    if len(text_no_space) > 0:
        # Remove all letters and digits
        text_no_alnum = re.sub(r'[a-zA-Z0-9]', '', text_no_space)
        
        # Calculate symbol ratio
        symbol_ratio = len(text_no_alnum) / len(text_no_space)
        
        # If 80% or more are symbols, mark as invalid
        if symbol_ratio >= 0.8:
            return 'invalid'
        # If less than 4 characters, mark as invalid
        if len(text_no_space) < 4:
            return 'invalid'

    
    # 7. Otherwise it's text
    return 'text'

#====================================================================================================
def add_row_number(df):
    # Add row_No column
    print("\nStep 5: Add row number column...")
    
    row_no = 1  # Start from row 1
    df.loc[0, 'row_No'] = row_no  # First element corresponds to row 1
    
    # Iterate through each row and check if it's a new row
    for i in range(1, len(df)):
        # Current row's center_y
        current_y = df.loc[i, 'left_bottom_y']
        
        # Previous row's center_y
        previous_y = df.loc[i-1, 'left_bottom_y']
        
        # Calculate the difference
        diff = abs(current_y - previous_y)
        
        # If difference < 20, consider same row; otherwise, assign a new row_No
        if diff < 20:
            df.loc[i, 'row_No'] = row_no
        else:
            row_no += 1  # For a new row, increment row_No by 1
            df.loc[i, 'row_No'] = row_no
    
    # Convert to integer type
    df['row_No'] = df['row_No'].astype(int)
    return df

#====================================================================================================
def add_priority_column(df):
    """
    Higher priority means closer to target content
    
    Rule:
    - Find first row where mark='header' → row_head
    - Find first row where text contains 'total' → row_tail
    - Rows where row_head <= row_No < row_tail → mark as 1
    - Other rows → mark as 0
    """

    print("\nStep 6: Add priority column...")
    
    # Initialize priority to 0 
    df['priority'] = 0
    
    # Find row_head (first header row)
    header_rows = df[df['mark'] == 'header']
    
    if len(header_rows) > 0:
        row_head = header_rows['row_No'].min()
    else:
        print("❌ No header found, using row_No 0 as row_head")
        row_head = 0
    
    # Find row_tail (first row with 'total' in text)
    total_rows = df[df['text'].str.lower().str.contains('total', na=False)]
    
    if len(total_rows) > 0:
        row_tail = total_rows['row_No'].min()
    else:
        print("❌ No 'total' found, using last row as row_tail")
        row_tail = df['row_No'].max() + 1
    
    # Set priority to 1 for rows between header and total
    # Rows where row_head <= row_No < row_tail → mark as 1
    # Other rows → mark as 0
    df['priority'] = ((df['row_No'] >= row_head) & (df['row_No'] < row_tail)).astype(int)
    
    return df
    
#====================================================================================================    
def add_column_number(df):
    """
    Add 'col_No' column based on x position.
    Rule: If horizontal overlap ratio > 70%, consider as same column.
    """
    print("\nStep 7: Add column number column...")
    # Sort by x (left to right), then y (top to bottom)
    df_sorted = df.sort_values(['left_top_x', 'left_top_y']).reset_index()

    df_sorted['col_No'] = 0
    curr_col_No = 1
    # Initialize current column x-range
    curr_min_x = df_sorted.loc[0, 'left_top_x']
    curr_max_x = df_sorted.loc[0, 'right_top_x']

    for idx, row in df_sorted.iterrows():
        left_x = row['left_top_x']
        right_x = row['right_top_x']
        width = right_x - left_x

        # Compute intersection with current column
        intersect_len = max(0, min(curr_max_x, right_x) - max(curr_min_x, left_x))
        overlap_ratio = max(intersect_len / width, intersect_len / (curr_max_x - curr_min_x))

        if overlap_ratio < 0.7:
            # Not enough overlap → new column
            curr_col_No += 1
            curr_min_x = left_x
            curr_max_x = right_x
        else:
            # Update current column range
            curr_min_x = min(curr_min_x, left_x)
            curr_max_x = max(curr_max_x, right_x)

        df_sorted.at[idx, 'col_No'] = curr_col_No

    # Restore original df order
    df_sorted = df_sorted.sort_values('index').drop(columns='index')
    df_sorted['col_No'] = df_sorted['col_No'].astype(int)

    return df_sorted

#====================================================================================================
# Update category text type: mark as "Unit"
def mark_units(df):
    """
    Mark units directly in the 'mark' column

    Rules::
    1. Directly matches common unit labels
    2. Number followed by common unit labels
    3. Pure number in the same column as header "Qty"
    """
    print("\nStep 8: Mark units...")
    
    # Common units
    unit_keywords = [
        # Counting
        'PC', 'PCS', 'PIECE', 'PIECES', 'EA', 'EACH', 'UNIT',
        
        # Weight
        'KG', 'G', 'GM', 'GRAM', 'LB', 'POUND', 'OZ', 'OUNCE',
        
        # Volume
        'L', 'ML', 'LITER', 'GAL', 'GALLON', 'FL OZ',
        
        # Packaging
        'BOX', 'BAG', 'BTL', 'BOTTLE', 'CAN', 'PKT', 'PACKET',
        'CTN', 'CARTON', 'JAR', 'TIN', 'TUBE',
        
        # Other
        'PAIR', 'SET', 'ROLL', 'PACK', 'BUNCH', 'DOZEN', 'DZ']
    
    # Find Qty column if exists
    qty_headers = df[(df['mark'] == 'header') & 
                    (df['text'].str.upper().str.contains('QTY|QUANTITY', na=False))]
    
    qty_col = None
    if len(qty_headers) > 0:
        qty_col = qty_headers.iloc[0]['col_No']    
    
    # Check each row
    for idx in df.index:
        text = df.loc[idx, 'text']
        text_upper = text.upper().strip()
        
        is_unit = False
        
        # Rule 1: Keyword match
        if text_upper in unit_keywords:
            is_unit = True
        
        # Rule 2: Pattern match (e.g., "1 PC")
        if not is_unit:
            pattern = r'^\d+\.?\d*\s*(' + '|'.join(unit_keywords) + r')$'
            if re.match(pattern, text_upper):
                is_unit = True
        
        # Rule 3: Pure number in Qty column
        if not is_unit and qty_col is not None:
            current_col = df.loc[idx, 'col_No']
            
            if current_col == qty_col:
                text_clean = text.replace('.', '').replace(',', '')
                if text_clean.isdigit():
                    try:
                        num = float(text)
                        if 0 < num < 1000:
                            is_unit = True
                    except:
                        pass
        
        # Update mark to 'unit' if identified
        if is_unit:
            df.loc[idx, 'mark'] = 'unit'
            
    return df

#====================================================================================================
# Update priority (mark as 2)
def update_priority_marks(df):
    """
    Update priority marks with additional rules

    Rules:
    1. If code and text are in same row or adjacent rows in same column，increase text's priority to 2
    1: If a code and text are adjacent in the same row or column, increase text's priority to 2
    3. If text is in same column as 'item' header，increase text's priority to 2
    """
    print("\nStep 9: Update priority marks...")
    
    # Keywords that indicate summary/total amounts
    total_keywords = [
        'amount', 'sum', 'balance', 'due', 'payable', 'net', 'gross']
    
    # Rule 1: Code + Text in same/adjacent rows in same column
    for idx, row in df.iterrows():
        if row['mark'] == 'text' and row['priority'] == 1:
            text_row = row['row_No']
            text_col = row['col_No']
            
            for idx2, row2 in df.iterrows():
                if row2['mark'] == 'code':
                    code_row = row2['row_No']
                    code_col = row2['col_No']
                    
                    same_row = (text_row == code_row)
                    same_col = (text_col == code_col)
                    adjacent_row = abs(text_row - code_row) == 1
                    
                    if same_row or (same_col and adjacent_row):
                        df.loc[idx, 'priority'] = 2
                        break
    
    # Rule 2: Price + Total keywords
    for idx, row in df.iterrows():
        if row['mark'] == 'price':
            price_col = row['col_No']
            price_row = row['row_No']
            
            for idx2, row2 in df.iterrows():
                text_lower = row2['text'].lower()
                
                contains_keyword = any(kw in text_lower for kw in total_keywords)
                
                if contains_keyword and row2['col_No'] == price_col:            
                    df.loc[idx, 'priority'] = 2
                    break
    
    # Rule 3: Text in same column as 'item' header
    # Find 'item' header column
    item_headers = df[(df['mark'] == 'header') & 
                     (df['text'].str.upper().str.contains('ITEM|DESCRIPTION|DESC|PRODUCT|NAME', na=False))]
    
    if len(item_headers) > 0:
        item_col = item_headers.iloc[0]['col_No']
        
        # Mark all text items in this column as priority=2
        for idx, row in df.iterrows():
            if row['mark'] == 'text' and row['col_No'] == item_col and row['priority'] == 1:
                df.loc[idx, 'priority'] = 2      
                text_row = row['row_No']
    else:
        print("❌No 'item' header found")
        
    return df

#====================================================================================================
def calculate_row_diff(df):
    """
    Calculate row difference between "price" and "text"
    Used to match extracted items with their corresponding amounts
    
    Logic:
    1. Count the number of "price" and "text" entries to find unmatched extra rows
    2. For the type with more entries, calculate row difference to nearest row of the other type
    3. Store row differences in row_diff column
    """
    print("\nStep 10: Calculate row differences...")
    
    # Filter priority=2 and mark in ['text', 'price'] 
    filtered = df[(df['priority'] == 2) & 
                  (df['mark'].isin(['text', 'price']))]
    
    # Count text and price
    text_count = len(filtered[filtered['mark'] == 'text'])
    price_count = len(filtered[filtered['mark'] == 'price'])
    
    # Initialize row_diff column
    filtered['row_diff'] = None
    
    # # Count of extra rows that need to be deleted
    delete_rows_count = 0

    # Compare row counts of price and text to set up data for deleting useless rows later
    # If the row count of price is greater: use the row of each price to identify the corresponding text with the nearest row
    if price_count > text_count:     
        search_type = 'price'
        target_type = 'text'
        delete_rows_count = price_count - text_count
        
    # If the row count of text is greater: use the row of each text to identify the corresponding price with the nearest row
    elif text_count > price_count:      
        search_type = 'text'
        target_type = 'price'
        delete_rows_count = text_count - price_count
    
    # Equal count, defaulting to use the row of each price to identify the corresponding text with the nearest row
    else:
        search_type = 'price'
        target_type = 'text'

    
    for idx in filtered.index:
        # Check if this is the search type
        if filtered.loc[idx, 'priority'] == 2 and filtered.loc[idx, 'mark'] == search_type:
            current_row = filtered.loc[idx, 'row_No']
            #current_text = filtered.loc[idx, 'text']
            
            # Find nearest target type
            min_diff = float('inf')
            nearest_target = None
            
            for idx2 in filtered.index:
                if filtered.loc[idx2, 'priority'] == 2 and filtered.loc[idx2, 'mark'] == target_type:
                    target_row = filtered.loc[idx2, 'row_No']
                    diff = abs(target_row - current_row)
                    
                    # Update if closer
                    if diff < min_diff:
                        min_diff = diff
                        nearest_target = {
                            'idx': idx2,
                            'row': target_row,
                            'text': filtered.loc[idx2, 'text']
                        }
            
            # Store the difference
            if nearest_target:
                # Calculate difference
                actual_diff = nearest_target['row'] - current_row
                filtered.loc[idx, 'row_diff'] = actual_diff
            else:
                filtered.loc[idx, 'row_diff'] = None
    return filtered, delete_rows_count
    
#====================================================================================================
def delete_by_row_diff(df, delete_rows_count):
    """
    Delete extra rows based on row_diff distribution
    
    Logic:
    1. Count row_diff value distribution
    2. Delete rows with least frequent and largest absolute difference
    3. Repeat until deleted delete_rows_count rows
    """
    print("\nStep 11: Delete rows based on row_diff...")
    
    # Get rows that have row_diff
    rows_with_diff = df[df['row_diff'].notna()].copy()
    
    if len(rows_with_diff) == 0:
        print("❌ No rows with row_diff to delete")
        print("="*70 + "\n")
        return df
        
    # Track deleted indices
    to_delete_indices = []
    deleted_count = 0
    
    # Loop until we've deleted enough rows
    while deleted_count < delete_rows_count:
        # Get current row_diff distribution (excluding already marked for deletion)
        remaining = rows_with_diff[~rows_with_diff.index.isin(to_delete_indices)]
        
        if len(remaining) == 0:
            print(f"\n❌ No more rows to delete (deleted {deleted_count}/{delete_rows_count})")
            break
        
        # Count distribution
        diff_counts = remaining['row_diff'].value_counts()
        
        # Find minimum count
        min_count = diff_counts.min()

        # Get all row_diff values with minimum count
        min_count_diffs = diff_counts[diff_counts == min_count].index.tolist()
       
        # Among these, find the one with largest absolute value
        max_abs_diff = max(min_count_diffs, key=abs)
        
        # Get rows with this row_diff value
        rows_to_delete = remaining[remaining['row_diff'] == max_abs_diff]

        # Delete these rows (or as many as needed)
        for idx in rows_to_delete.index:
            if deleted_count >= delete_rows_count:
                break
            
            to_delete_indices.append(idx)
            deleted_count += 1
    
    # Actually delete from DataFrame
    df = df.drop(to_delete_indices).reset_index(drop=True)
    return df

#====================================================================================================
def create_matched_dataframe(df):
    """
    Create final DataFrame with matched items and prices
    Add row_match column; rows with the same row_match form a pair
    
    Columns: item, price
    """
    print("\nStep 12: Create matched DataFrame...")

    # Add row_match column
    # If row_diff is None, use row_No directly; otherwise, use row_No + row_diff
    df['row_match'] = df.apply(lambda row: row['row_No'] if pd.isna(row['row_diff']) 
                                                        else row['row_No'] + row['row_diff'], axis=1).astype(int)
    
    # Get unique row_match values
    unique_matches = df['row_match'].unique()
    unique_matches = sorted(unique_matches)
    
    # Store matched pairs
    matched_list = []
    
    # Process each row_match value
    for match_value in unique_matches:
        # Get all rows with this row_match
        matched_rows = df[df['row_match'] == match_value]
        
        # Find text and price
        texts = matched_rows[matched_rows['mark'] == 'text']
        prices = matched_rows[matched_rows['mark'] == 'price']
        
        # Only add if both text and price exist
        if len(texts) > 0 and len(prices) > 0:
            # Get first text and first price
            item_text = texts.iloc[0]['text']
            price_text = prices.iloc[0]['text']
            
            # Add to list
            matched_list.append({
                'item': item_text,
                'price': price_text
            })
        else:
            if len(texts) == 0:
                print(f"No text found")
            elif len(prices) == 0:
                print(f"No price found (has text: '{texts.iloc[0]['text']}')")
    
    # Create final DataFrame
    final_df = pd.DataFrame(matched_list)
    
    print("Final matched DataFrame:")
    print(f"Total matched pairs: {len(final_df)}")
    print("="*70 + "\n")
    
    return final_df

# PART 4: Interactive Visualization

In [392]:
# Count letters
def count_letters(text):
    if text is None:
        return 0
    c = 0
    for ch in str(text):
        if ('a' <= ch <= 'z') or ('A' <= ch <= 'Z'):
            c += 1
    return c

# Price validation: numeric or decimal (up to two decimal places)
def validate_price(price_text):
    if price_text is None:
        return False, None
    s = str(price_text).strip()
    if s == "":
        return False, None
    if re.match(r'^\d+(\.\d{1,2})?$', s):
        try:
            return True, float(s)
        except:
            return False, None
    return False, None

#====================================================================================================
# Main function for interactive page
def create_interactive_viewer(receipt):
    """
    - receipt: dict, keys: 'date' (optional), 'items' (list of dicts with 'Item' and 'Price')
    """
    # Import data
    items = receipt.get("items", []) or []
    date = receipt.get("date", "Unknown")
    
    if len(items) == 0:
        df = pd.DataFrame(columns=["Item", "Price"])
        print("⚠️ No items detected, showing empty template")
    else:
        # Only these two columns
        temp = pd.DataFrame(items)
        for col in ["Item","Price"]:
            if col not in temp.columns:
                temp[col] = ""
        df = temp[["Item","Price"]].copy()
    
    # Deep copy the original data for reset (to prevent later modifications from affecting the original)
    original_df = df.copy(deep=True)
    
    # Receipt ID counter (by date)
    if not hasattr(create_interactive_viewer, "counter"):
        create_interactive_viewer.counter = {}
    if date not in create_interactive_viewer.counter:
        create_interactive_viewer.counter[date] = 0
    create_interactive_viewer.counter[date] += 1
    receipt_id = create_interactive_viewer.counter[date]
    
    # UI
    table_output = widgets.Output()
    total_output = widgets.Output()
    status_output = widgets.Output()
    
    row_input = widgets.IntText(value=1, description='Row:', min=1)
    item_input = widgets.Text(description='Item:', placeholder='Item name (max 30 letters)')
    price_input = widgets.Text(description='Price:', placeholder='e.g. 12.50')
    
    add_btn = widgets.Button(description='Add', button_style='success')
    update_btn = widgets.Button(description='Update', button_style='info')
    delete_btn = widgets.Button(description='Delete', button_style='danger')
    reset_btn = widgets.Button(description='Reset', button_style='warning')
    export_btn = widgets.Button(description='Export CSV', button_style='primary')
    
    # Calculate total price
    def calculate_total(df_now):
        total = 0.0
        for i, row in df_now.iterrows():
            ok, v = validate_price(str(row["Price"]).strip())
            if ok:
                total += v
        return total
    
    # Refresh table display
    def refresh_table():
        with table_output:
            clear_output()
            tmp = df.copy()
            tmp.index = range(1, len(tmp) + 1)
            display(tmp)
    # Refresh total price display
    def refresh_total():
        with total_output:
            clear_output()
            total = calculate_total(df)
            print("TOTAL: $%.2f" % total)
    
    # Initially display header information
    print("\n============================")
    print("RECEIPT VIEWER")
    print("============================")
    print("Date       :", date)
    print("Receipt ID :", receipt_id)
    print("Items      :", len(df))
    print("============================\n")

    # Refresh the table and total display  
    refresh_table()
    display(table_output)
    refresh_total()
    display(total_output)

    #========================================================
    # Button function  
    def on_add(btn):
        with status_output:
            clear_output()
            item = item_input.value or ""
            price = price_input.value or ""
            
            if item.strip() == "":
                print("❌ Please enter an Item.")
                return
            if price.strip() == "":
                print("❌ Please enter a Price.")
                return
            
            letters = count_letters(item)
            if letters > 30:
                print("❌ Item has %d letters. Maximum allowed is 30." % letters)
                return
            
            ok, val = validate_price(price)
            if not ok:
                print("❌ Invalid price format. Use digits or decimal like '12' or '12.50'.")
                return
            
            # Add new row
            df.loc[len(df)] = [item.strip(), "%.2f" % val]
            # Update row_input maximum
            row_input.max = max(1, len(df))

            # Refresh the table and total display  
            refresh_table()
            refresh_total()
            print("✅ Item added.")
            # Clear input
            item_input.value = ""
            price_input.value = ""
    
    def on_update(btn):
        with status_output:
            clear_output()
            idx = row_input.value - 1
            if idx < 0 or idx >= len(df):
                print("❌ Invalid row number.")
                return
            
            new_item = item_input.value or ""
            new_price = price_input.value or ""
            
            # If input is not empty, validate and update; otherwise keep original value
            if new_item.strip() != "":
                letters = count_letters(new_item)
                if letters > 30:
                    print("❌ Item has %d letters. Maximum allowed is 30." % letters)
                    return
                df.at[idx, "Item"] = new_item.strip()
            
            if new_price.strip() != "":
                ok, val = validate_price(new_price)
                if not ok:
                    print("❌ Invalid price format.")
                    return
                df.at[idx, "Price"] = "%.2f" % val

            # Refresh the table and total display  
            refresh_table()
            refresh_total()
            print("✅ Row updated.")
            item_input.value = ""
            price_input.value = ""
    
    def on_delete(btn):
        with status_output:
            clear_output()
            idx = row_input.value - 1
            if idx < 0 or idx >= len(df):
                print("❌ Invalid row number.")
                return
            # Delete in place and reset index
            df.drop(idx, inplace=True)
            df.reset_index(drop=True, inplace=True)
            row_input.max = max(1, len(df))
            if row_input.value > row_input.max:
                row_input.value = row_input.max

            # Refresh the table and total display  
            refresh_table()
            refresh_total()
            print("✅ Row deleted.")
    
    def on_reset(btn):
        nonlocal df  # Use the outer df variable to allow rebinding, preventing crashes when resetting after export
        with status_output:
            clear_output()
            
            # Restore df to the original snapshot (deep copy to avoid shared references)
            df = original_df.copy(deep=True).reset_index(drop=True)

            # Update row_input maximum value and adjust current value if needed
            row_input.max = max(1, len(df))
            if row_input.value > row_input.max:
                row_input.value = row_input.max

            # Refresh the table and total display    
            refresh_table()
            refresh_total()
            print("✅ Data reset.")
    
    def on_export(btn):
        # When exporting, do not rebind df; just read and write to file
        with status_output:
            clear_output()
            total = calculate_total(df)
            out = []
            for i, row in df.iterrows():
                price_s = str(row["Price"]).strip()
                ok, val = validate_price(price_s)
                if not ok:
                    val = 0.0
                out.append({
                    "Date": date,
                    "ID": receipt_id,
                    "Item": row["Item"],
                    "Cost": "%.2f" % val,
                    "Total": "%.2f" % total
                })
            out_df = pd.DataFrame(out)
            fname = "receipt_%s_id%s.csv" % (date.replace("/", "-"), receipt_id)
            out_df.to_csv(fname, index=False)
            print("✅ Exported:", fname)
    
    # Connect button
    add_btn.on_click(on_add)
    update_btn.on_click(on_update)
    delete_btn.on_click(on_delete)
    reset_btn.on_click(on_reset)
    export_btn.on_click(on_export)
    
    # Display widget
    display(row_input)
    display(item_input)
    display(price_input)
    display(widgets.HBox([add_btn, update_btn, delete_btn, reset_btn, export_btn]))
    display(status_output)
    
    print("\nInstructions:")
    print("1) Add: both Item and Price required.")
    print("2) Update: leave a field empty to keep original.")
    print("3) Price format: digits or decimal, e.g. 12 or 12.50")
    print("4) Item: non-empty and max 30 letters.")
    print("========================")


# PART 5: Main Processing Function

In [393]:
#Process 1: Get data – Image preprocessing and OCR recognition
def process1_getdata(image_path):
    print("\n" + "="*70)
    print(f"Processing: {os.path.basename(image_path)}")
    print("="*70)
    
    # Preprocess
    print("Process 1: Get data...")
    print("Step 1: Preprocessing...")
    preprocessed = preprocess_image(image_path)
    if preprocessed is None:
        print("❌ Preprocessing failed!")
        return None
    print("✅ Preprocessing done")
    
    # OCR
    print("\nStep 2: OCR Recognition...")
    ocr = initialize_ocr()
    ocr_result = perform_ocr(ocr, image_path)

    if len(ocr_result) == 0:
        print("❌ No text detected!")
        return None
    print(f"✅ Detected {len(ocr_result)} text regions")

    # Organize
    print("\nStep 3: Organizing data...")
    df_ocr_result = organize_ocr_results(ocr_result)
    print("✅ Data organized")
    return df_ocr_result

# Process 2: Extracting information
def process2_get_main_information(df_ocr_result):     
    print("="*70)  
    print("Process 2: Extracting information...")
    print("Step 4: Add mark column...")

    # Roughly label text categories
    df_ocr_result['mark'] = df_ocr_result['text'].apply(classify_text_type)

    # Get date
    date_rows = df_ocr_result[df_ocr_result['mark'] == 'date']
    if len(date_rows) > 0:
        date_str = date_rows['text'].iloc[0]
        date_str = extract_date(date_str)
    else:
        date_str = datetime.now().strftime("%m/%d/%Y")
    
    # Add row number
    df_mark_data =  add_row_number(df_ocr_result)

    # Identify valid table content and set priority to 1
    df_mark_data = add_priority_column(df_mark_data)
    df_valid_data = df_ocr_result[df_mark_data['priority'] == 1].copy()

    # Add column number
    df_valid_data = add_column_number(df_valid_data)

    # Identify text labeled as "Unit"
    df_valid_data = mark_units(df_valid_data)

    # Further identify valid information and set priority to 2
    df_valid_data = update_priority_marks(df_valid_data)

    # Calculate row differences of target information for subsequent matching
    df_goal_data,delete_rows_count = calculate_row_diff(df_valid_data)
    
    # Delete unmatched extra information
    df_goal_data = delete_by_row_diff(df_goal_data, delete_rows_count)

    # Retrieve matched information
    df_matched_goal_data = create_matched_dataframe(df_goal_data)

    print("✅ Target information extraction completed")    
    return date_str, df_mark_data, df_valid_data, df_goal_data, df_matched_goal_data

    
# Process 3: Interactive visualization page
def process3_display(date, df):
    print("\nProcess 3: Creating viewer...")
    receipt = {
    'date': date,
    'items': df.rename(
                columns={'item': 'Item', 'price': 'Price'}).to_dict('records')}
    create_interactive_viewer(receipt)
    return "Thank you!"


# Entry point ⬇️

In [394]:
# Process 1: Get data – Image preprocessing and OCR recognition

# Input image path
# image_path = 'X00016469612.jpg'
image_path = 'X00016469672.jpg'
# image_path = 'X00016469622.jpg'

# Get raw OCR data and save to DataFrame
df_ocr_result = process1_getdata(image_path)

# Print raw data
df_ocr_result


Processing: X00016469672.jpg
Process 1: Get data...
Step 1: Preprocessing...


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.


✅ Preprocessing done

Step 2: OCR Recognition...


Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/en_PP-OCRv5_mobile_rec`.


✅ Detected 15 text regions

Step 3: Organizing data...
Organize OCR data...
✅ Data organized


,text,confidence,left_top_x,left_top_y,right_top_x,right_top_y,left_bottom_x,left_bottom_y,right_bottom_x,right_bottom_y
0,tan chay yee,0.991900,97,0,316,0,316,33,97,33
1,SOON HUAT MACHINERY ENTERPRISE,0.984482,19,53,431,61,430,88,18,80
2,(JM0352019-K),0.974050,160,79,294,82,293,110,159,107
3,NO.53 JALAN PUTRA 1.,0.971054,127,107,327,110,326,135,126,131
4,TAMAN SRI PUTRA.,0.994149,142,133,312,135,312,159,142,157
...,...,...,...,...,...,...,...,...,...,...
86,327.00,0.999899,354,1082,413,1082,413,1103,354,1103
87,CASH :,0.990495,244,1113,323,1110,324,1135,244,1138
88,327.00,0.999916,355,1114,414,1114,414,1134,355,1134
89,Change :,0.991664,226,1139,322,1135,323,1160,227,1163


In [395]:
# Process 2: Extracting information
'''
date: extracted date
df_mark_data: data with category labels
df_valid_data: first-stage filtered data (priority = 1)
df_goal_data: second-stage filtered data (priority = 2) with unmatched rows removed
df_matched_goal_data: final data after information matching 
'''
# Get data from different stages
date, df_mark_data, df_valid_data, df_goal_data, df_matched_goal_data = process2_get_main_information(df_ocr_result)

# Print final data
df_matched_goal_data

Process 2: Extracting information...
Step 4: Add mark column...

Step 5: Add row number column...

Step 6: Add priority column...

Step 7: Add column number column...

Step 8: Mark units...

Step 9: Update priority marks...

Step 10: Calculate row differences...

Step 11: Delete rows based on row_diff...

Step 12: Create matched DataFrame...
Final matched DataFrame:
Total matched pairs: 8

✅ Target information extraction completed


,item,price
0,REPAIR ENGINE POWER SPRAYER (1UNIT),80.00
1,GIANT 606 OVERFLOW ASSY,160.00
2,ENGINE OIL,17.00
3,GREASE FOR TOOLS 40ML (AKODA),10.00
4,EY20 PLUG CHAMPION,6.00
5,STARTER TALI,8.00
6,EY20 STARTER HANDLE,10.00
7,HD40 1L COTIN,36.00


In [396]:
# Process 3: Interactive visualization page

process3_display(date, df_matched_goal_data)


Process 3: Creating viewer...

RECEIPT VIEWER
Date       : 01/11/2019
Receipt ID : 1
Items      : 8



Output()

Output()

IntText(value=1, description='Row:')

Text(value='', description='Item:', placeholder='Item name (max 30 letters)')

Text(value='', description='Price:', placeholder='e.g. 12.50')

Output()


Instructions:
1) Add: both Item and Price required.
2) Update: leave a field empty to keep original.
3) Price format: digits or decimal, e.g. 12 or 12.50
4) Item: non-empty and max 30 letters.


'Thank you!'